In [1]:
#功能：输入肺音的时频矩阵，用CNN判断其类别
#0：SR  1：MR  2：VB  3：VR  4：WR

import torch
from torch import nn
import pandas as pd
import time
import copy
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import xlrd
from torch.autograd import Variable
from torch.utils.data import Dataset  
from torch.utils.data import DataLoader 
from torchvision import datasets
import torch.utils.data as Data

In [2]:
class lungDataset(Dataset):#
    def __init__(self,inputs,sig_num,in_width,in_height):  
        xy1=np.loadtxt(open(inputs,"rb"),delimiter=",",skiprows=0)
        
        x=np.zeros([sig_num,in_width,in_height]) 
        y=np.zeros([sig_num])
        for i in range(sig_num):  
            x[i,:,:]=xy1[in_height*i:in_height*(i+1),0:256].T
            
            y[i]=xy1[in_height*i,256]#LABEL
        
        x=x.astype('float32') 
        y=y.astype('int64') 
        x=torch.from_numpy(x);y=torch.from_numpy(y)
        x=x[:,np.newaxis,:,:]
        x = Variable(x,requires_grad=True);
#         x=x.cuda(0);y=y.cuda(0)
        self.x_data = x
        self.y_data = y
        self.len = x.shape[0] 
        
    def __getitem__(self, index):    
        return self.x_data[index], self.y_data[index]
 
    def __len__(self): #
        return self.len


In [3]:
in_width=256
in_height=128
batch_size_train=4
batch_size_validation=4
sig_num_train=56
sig_num_validation=28

In [4]:
dataset = lungDataset(inputs="56肺时频矩阵.csv",sig_num=sig_num_train,in_width=in_width,
                          in_height=in_height) #
train_loader=DataLoader(dataset=dataset, batch_size=batch_size_train, shuffle=False, num_workers=0)

dataset=lungDataset(inputs="28肺时频矩阵.csv",sig_num=sig_num_validation,in_width=in_width,
                        in_height=in_height)
validation_loader=DataLoader(dataset=dataset,batch_size=batch_size_validation,shuffle=False,num_workers=0)



In [5]:
for step,(x,y) in enumerate(train_loader):
    pass
print("step：",step+1)
print(x.dtype) 
print(x.shape)  
print(type(x))  
print(x.requires_grad) 
print(y.dtype) 
print(y.shape)  
print(type(y))  
print(y.requires_grad) 

for step,(x,y) in enumerate(validation_loader):
    pass
print("step：",step+1)
print(x.dtype) 
print(x.shape) 
print(type(x))  
print(x.requires_grad) 
print(y.dtype) 
print(y.shape)  
print(type(y))  
print(y.requires_grad) 

step： 14
torch.float32
torch.Size([4, 1, 256, 128])
<class 'torch.Tensor'>
True
torch.int64
torch.Size([4])
<class 'torch.Tensor'>
False
step： 7
torch.float32
torch.Size([4, 1, 256, 128])
<class 'torch.Tensor'>
True
torch.int64
torch.Size([4])
<class 'torch.Tensor'>
False


In [6]:
class MyDCNN(torch.nn.Module):
    def __init__(self):
        super(MyDCNN, self).__init__()
        self.pooling1=torch.nn.AvgPool2d(kernel_size=2,stride=2)
        self.cnn10=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,
                              stride=1,padding=1,bias=False, dilation=2)
    
        self.cnn20=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,
                              stride=1,padding=1,bias=False, dilation=2)

        self.cnn30=nn.Conv2d(in_channels=32,out_channels=48,kernel_size=3,
                        stride=1,padding=1,bias=False, dilation=2)
        
        self.cnn40=nn.Conv2d(in_channels=48,out_channels=64,kernel_size=3,
                        stride=1,padding=1,bias=False, dilation=2)
        self.bn4=nn.BatchNorm2d(64)
        self.fc1=torch.nn.Linear(5376,128)
        self.fc2=torch.nn.Linear(128,64)
        self.fc3=torch.nn.Linear(64,8)
    def forward(self,x):
        batch_size=x.size(0)
        en=nn.functional.elu(x, alpha=1.0, inplace=False)
        
        en=torch.relu(self.cnn10(en))
        en =self.pooling1(en)
              
        en=torch.relu(self.cnn20(en))
        en =self.pooling1(en)
        
        en=torch.relu(self.cnn30(en))
        en =self.pooling1(en)
              
        en=torch.relu(self.cnn40(en))
        en=self.bn4(en)
        en =self.pooling1(en)
              
        en=en.view(batch_size,-1)
        en=self.fc1(en)
        en=self.fc2(en)
        en=self.fc3(en)
        return en
   
   

In [7]:
lr = 0.0001
model=MyDCNN()
# model.cuda()
print(model)
optimizer=torch.optim.Adam(model.parameters(),lr)
criterion = nn.CrossEntropyLoss()
# criterion.cuda()


MyDCNN(
  (pooling1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (cnn10): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2), bias=False)
  (cnn20): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2), bias=False)
  (cnn30): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2), bias=False)
  (cnn40): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2), bias=False)
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=5376, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)


In [8]:
train_loss_all=[]
train_acc_all=[]
validation_loss_all=[]
validation_acc_all=[]
num_epochs=301


In [9]:
print('Epoch: ',num_epochs,'迭代率',lr)
print('训练Batch:',batch_size_train,'验证Batch:',batch_size_validation)
print('训练集信号个数:',sig_num_train,'验证集信号个数:',sig_num_validation)
print('-'*50)

for epoch in range(num_epochs):
    start_time = time.time()
    torch.cuda.empty_cache()##t
    torch.backends.cudnn.benchmark = True 
    model.train() #
    train_corrects=0;validation_corrects=0
    train_num=0;validation_num=0
    
    for step,(b_x,b_y) in enumerate(train_loader):      
        xdata=b_x  #[batch,in_channels,256,128]
        output=model(xdata) 
        pre_lab=torch.argmax(output,1)  
        loss=criterion(output,b_y) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        
        loss+=loss.item()*b_x.size(0)              # 
        train_corrects+=torch.sum(pre_lab == b_y.data)   #
        train_num+=b_x.size(0) #
    
    train_loss_all.append(loss/train_num)
    train_acc_all.append(train_corrects.double().item()/train_num)
    print('Epoch: {}/{}'.format(epoch+1,num_epochs))
    print('训练集loss:{:.4f} ，训练集acc:{:.4f}，用时：{:.4f}s'.format(train_loss_all[-1],
                                                         train_acc_all[-1],(time.time() - start_time)))
       
    if epoch % 20 == 0:
        model.eval() #VAL
        corrects=0
        validation_num=0
        for step, (c_x,c_y) in enumerate(validation_loader, 0):
            output=model(c_x) 
            pre_lab=torch.argmax(output,1)  
            loss=criterion(output,c_y)
            loss+=loss.item()*c_x.size(0) 
            validation_corrects+=torch.sum(pre_lab == c_y.data)
            validation_num+=c_x.size(0) 
        
        validation_loss_all.append(loss/validation_num)
        validation_acc_all.append(validation_corrects.double().item()/validation_num)
        print('|','-'*80,'|')
        print('test_loss:{:.4f} ，test_acc:{:.4f}，time：{:.4f}s'.format(train_loss_all[-1],
                                                         train_acc_all[-1],(time.time() - start_time)))
        print('|','-'*80,'|')


Epoch:  301 迭代率 0.0001
训练Batch: 4 验证Batch: 4
训练集信号个数: 56 验证集信号个数: 28
--------------------------------------------------
Epoch: 1/301
训练集loss:0.1397 ，训练集acc:0.4464，用时：2.7825s
| -------------------------------------------------------------------------------- |
测试集loss:0.1397 ，测试集集acc:0.4464，用时：3.1699s
| -------------------------------------------------------------------------------- |
Epoch: 2/301
训练集loss:0.0628 ，训练集acc:0.8214，用时：2.6025s
Epoch: 3/301
训练集loss:0.0423 ，训练集acc:0.8750，用时：2.5938s
Epoch: 4/301
训练集loss:0.0241 ，训练集acc:0.9643，用时：2.9071s
Epoch: 5/301
训练集loss:0.0161 ，训练集acc:0.9821，用时：3.2191s
Epoch: 6/301
训练集loss:0.0113 ，训练集acc:0.9821，用时：2.9128s
Epoch: 7/301
训练集loss:0.0079 ，训练集acc:1.0000，用时：3.1118s
Epoch: 8/301
训练集loss:0.0058 ，训练集acc:1.0000，用时：3.7813s
Epoch: 9/301
训练集loss:0.0044 ，训练集acc:1.0000，用时：3.0079s
Epoch: 10/301
训练集loss:0.0034 ，训练集acc:1.0000，用时：3.6872s
Epoch: 11/301
训练集loss:0.0028 ，训练集acc:1.0000，用时：5.2688s
Epoch: 12/301
训练集loss:0.0023 ，训练集acc:1.0000，用时：4.0565s
Epoch: 13/301
训练集

KeyboardInterrupt: 

In [27]:
#保存模型
torch.save(model.state_dict(), 'my_dcnn.pt')

In [28]:
m_state_dict = torch.load('my_dcnn.pt')  #加载模型参数
new_m = MyDCNN()              #加载模型
new_m.load_state_dict(m_state_dict) #模型参数导入新模型

<All keys matched successfully>

In [29]:
start_time = time.time()
acc_all=[]
dataset=lungDataset(inputs="12肺时频矩阵.csv",sig_num=12,in_width=in_width,
                        in_height=in_height)
validation_loader=DataLoader(dataset=dataset,batch_size=6,shuffle=False,num_workers=0)
for step,(x,y) in enumerate(validation_loader):
    train_corrects=0
    output_test=new_m(x)
    pre_lab=torch.argmax(output_test,1)
    print('预测分类',pre_lab)
    print('真实分类',y)
    
    corrects=torch.sum(pre_lab == y.data) #正确个数 pre_lab == b_y.data
    num=x.size(0) #训练总数######计算经过一个epoch后在下训练集上的精度
    acc_all.append(corrects.double().item()/num)
    print('第{}轮: acc:{:.3f}，time：{:.4f}s'.format(step+1,
                                                     acc_all[-1],(time.time() - start_time)))
    print('-'*80)
       

预测分类 tensor([0, 1, 3, 3, 3, 2])
真实分类 tensor([0, 1, 3, 3, 3, 2])
第1轮: acc:1.000，time：0.4358s
--------------------------------------------------------------------------------
预测分类 tensor([4, 4, 4, 2, 1, 4])
真实分类 tensor([4, 4, 4, 2, 1, 4])
第2轮: acc:1.000，time：0.5047s
--------------------------------------------------------------------------------
